In [18]:
from transformers import LiltForTokenClassification, LayoutLMv3Processor
from PIL import Image, ImageDraw, ImageFont
import torch
 
# load model and processor from huggingface hub
model_id="philschmid/lilt-en-funsd"
model_id="philschmid/lilt-en-funsd"
model = LiltForTokenClassification.from_pretrained(model_id)
processor = LayoutLMv3Processor.from_pretrained(model_id)


Some weights of LiltForTokenClassification were not initialized from the model checkpoint at SCUT-DLVCLab/lilt-roberta-en-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OSError: SCUT-DLVCLab/lilt-roberta-en-base does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/SCUT-DLVCLab/lilt-roberta-en-base/tree/main' for available files.

In [17]:
# helper function to unnormalize bboxes for drawing onto the image
def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]
 
 
label2color = {
    "B-HEADER": "blue",
    "B-QUESTION": "red",
    "B-ANSWER": "green",
    "I-HEADER": "blue",
    "I-QUESTION": "red",
    "I-ANSWER": "green",
}
# draw results onto the image
def draw_boxes(image, boxes, predictions):
    width, height = image.size
    normalizes_boxes = [unnormalize_box(box, width, height) for box in boxes]
 
    # draw predictions over the image
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for prediction, box in zip(predictions, normalizes_boxes):
        if prediction == "O":
            continue
        draw.rectangle(box, outline="black")
        draw.rectangle(box, outline=label2color[prediction])
        draw.text((box[0] + 10, box[1] - 10), text=prediction, fill=label2color[prediction], font=font)
    return image
 
 
# run inference
# def run_inference(image, model=model, processor=processor, output_image=True):
#     # create model input
    
#     encoding = processor(image, return_tensors="pt")
    
#     #['id', 'tokens', 'bboxes', 'ner_tags', 'image']
#     del encoding["pixel_values"]
#     # run inference
#     outputs = model(**encoding)
#     predictions = outputs.logits.argmax(-1).squeeze().tolist()
#     # get labels
#     labels = [model.config.id2label[prediction] for prediction in predictions]
#     if output_image:
#         return draw_boxes(image, encoding["bbox"][0], labels)
#     else:
#         return labels


def run_inference(example, model=model, processor=processor, output_image=True):
    # create model input
    
    image = example['image']
    processor.feature_extractor.apply_ocr = False
    encoding = processor(image, example['tokens'], boxes=example['bboxes'], word_labels=example['ner_tags'],
                           padding="max_length", truncation=True, return_tensors="pt")

    print(encoding.keys())
    #['id', 'tokens', 'bboxes', 'ner_tags', 'image']
    del encoding["pixel_values"]
    # run inference
    outputs = model(**encoding)
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    # get labels
    labels = [model.config.id2label[prediction] for prediction in predictions]
    if output_image:
        return draw_boxes(image, encoding["bbox"][0], labels)
    else:
        return labels
 
dataset_id ="nielsr/funsd-layoutlmv3"
from datasets import load_dataset
dataset = load_dataset(dataset_id)



# run_inference(dataset["test"][34]["image"])
run_inference(dataset["test"][34])



dict_keys(['input_ids', 'attention_mask', 'bbox', 'labels', 'pixel_values'])


IndexError: Target 3 is out of bounds.

In [9]:
dataset["test"][34].keys()

dict_keys(['id', 'tokens', 'bboxes', 'ner_tags', 'image'])

In [11]:
type(dataset["test"][34]["image"])

PIL.PngImagePlugin.PngImageFile